In [54]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectPercentile,f_classif
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
from sklearn.metrics import accuracy_score, classification_report
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv


In [2]:
df_train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
df_train_sp = df_train.sample(frac=0.7, random_state=1, replace=False)
df_vald_sp = df_train[~df_train.id.isin(df_train_sp.id)]

In [5]:
df_train_sp.shape[0]

5329

In [6]:
df_vald_sp.shape[0]

2284

In [7]:
df_test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
df_subm = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')
df_subm.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [10]:
df_train.shape[0]

7613

In [ ]:
df_train[(df_train.target == 0)]["text"].tolist()

In [ ]:
df_train[(df_train.target == 0)]["text"].tolist()

In [ ]:
df_train.keyword.isnull().sum()

In [ ]:
df_test.keyword.isnull().sum()

In [ ]:
df_tra

In [ ]:
df_train.text.head().tolist()

In [47]:
def tokenize(text):    
    url_pat = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    found_urls = re.findall(url_pat, text)
    for url in found_urls:
        text = text.replace(url, "urlplaceholder")   
    
    text = re.sub("[=.<>0-9-]", " ", text)

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_token = lemmatizer.lemmatize(tok).lower().strip()
        clean_token = lemmatizer.lemmatize(clean_token, pos='v')  
        clean_tokens.append(clean_token)
    return clean_tokens

In [12]:
t = 'I wanted to set Chicago ablaze with my preaching... But not my hotel'

In [13]:
tokenize(t)

['i',
 'want',
 'to',
 'set',
 'chicago',
 'ablaze',
 'with',
 'my',
 'preach',
 'but',
 'not',
 'my',
 'hotel']

In [ ]:
nltk.pos_tag(tokenize(t))

In [ ]:
df_train[df_train.target == 1]

In [ ]:
df_train_fake = df_train[df_train.target == 0]

In [ ]:
df_train_fake.shape[0]

In [ ]:
df_train_fake[~df_train_fake.keyword.isna()]

In [ ]:
df_train_real = df_train[df_train.target == 1]

In [ ]:
df_train_real.shape[0]

In [ ]:
df_train_real[~df_train_real.keyword.isna()]

In [ ]:
df_train_sp[df_train_sp.target == 0]

In [ ]:
df_train_sp[df_train_sp.target == 1].iloc[0]["text"]

In [14]:
class DataFrameColumnExtracter(TransformerMixin):

    def __init__(self, column, conv_bin=False):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):        
        return X[self.column]

In [15]:
class BinConverter(BaseEstimator,TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return pd.DataFrame(pd.Series(X).notnull())

In [16]:
class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

In [48]:
tfvec = TfidfVectorizer(tokenizer=tokenize)

In [49]:
tfidf = tfvec.fit_transform(df_train_sp['text'])

In [50]:
clf = LogisticRegression()

In [51]:
clf.fit(tfidf, df_train_sp.target)

LogisticRegression()

In [52]:
tfvec.get_feature_names()[1]

'#'

In [41]:
tfvec.vocabulary_

{'goulburn': 6548,
 'man': 9318,
 'henry': 7014,
 'van': 15534,
 'bilsen': 2384,
 'missing': 9744,
 'emergency': 5124,
 'services': 13183,
 'are': 1691,
 'searching': 13074,
 'for': 5966,
 'who': 16045,
 'disappeared': 4532,
 'from': 6117,
 'his': 7102,
 'û_': 16825,
 'http': 7286,
 'co': 3500,
 'z99pkjztrp': 16678,
 'the': 14565,
 'things': 14639,
 'we': 15925,
 'fear': 5670,
 'most': 9913,
 'in': 7587,
 'organizations': 10852,
 'fluctuations': 5915,
 'disturbances': 4593,
 'imbalances': 7553,
 'primary': 11658,
 'sources': 13709,
 'of': 10645,
 'creativity': 3891,
 'margaret': 9356,
 'wheatley': 16019,
 'tsunami_esh': 15081,
 'hey': 7031,
 'esh': 5302,
 'potus': 11542,
 'you': 16593,
 'until': 15368,
 'drown': 4807,
 'by': 2865,
 'water': 15885,
 'entering': 5225,
 'lungs': 9174,
 'being': 2249,
 'alive': 1405,
 'has': 6885,
 'caused': 3113,
 'this': 14648,
 'great': 6602,
 'country': 3822,
 'to': 14801,
 'fall': 5575,
 'shit': 13303,
 'because': 2205,
 're': 12185,
 'pussy': 11842,


In [ ]:
selector.fit(X_all, y_all)

In [53]:
for i in range(16867):
    imp = clf.coef_[0][i]
    feat = tfvec.get_feature_names()[i]
    if imp > 0.7:
        print(feat)

#
:
abc
accident
after
aircraft
airplane
airport
allow
american
amid
another
area
arrest
at
atomic
attack
aware
bigger
bomb
bomber
bridge
build
burn
by
calgary
california
car
case
casualty
catch
cause
central
charge
city
civilian
close
collapse
collision
confirm
continue
costlier
country
county
crash
crew
crisis
cut
damage
dead
death
debris
declare
derailment
devastation
disaster
drought
due
during
dust
earthquake
east
ebola
evacuate
evacuation
event
explosion
family
find
fire
flood
forest
from
gas
global
hail
hailstorm
hazardous
heat
helicopter
hiroshima
home
horror
hostage
hundred
hurricane
hwo
in
incident
injure
isi
israel
israeli
issue
japan
kill
land
latest
legionnaire
libya
lightning
live
lose


KeyboardInterrupt: 

In [172]:
keywords_pipe = make_pipeline(
    DataFrameColumnExtracter('keyword'), 
    BinConverter()
    
)

location_pipe = make_pipeline(
    DataFrameColumnExtracter('location'), 
    BinConverter()
)

text_pipe = make_pipeline(
    DataFrameColumnExtracter('text'), 
    CountVectorizer(tokenizer=tokenize),
    TfidfTransformer(),
#     SelectPercentile(f_classif, percentile=95)
)



feature_union = make_union(text_pipe, keywords_pipe, location_pipe)

In [173]:
# pipeline = Pipeline([
#     ('vect', CountVectorizer(tokenizer=tokenize)),
#     ('tfidf', TfidfTransformer()),
#     ('clf', LogisticRegression())
# ])
    
pipeline = Pipeline([
    ('features', feature_union),
#     ('dense', DenseTransformer()),
#     ('clf', GaussianNB())
    ('clf', LogisticRegression())
])

In [174]:
pipeline.fit(df_train_sp, df_train_sp["target"])

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('pipeline-1',
                                                 Pipeline(steps=[('dataframecolumnextracter',
                                                                  <__main__.DataFrameColumnExtracter object at 0x7fc69f8c3a50>),
                                                                 ('countvectorizer',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x7fc6a6967e60>)),
                                                                 ('tfidftransformer',
                                                                  TfidfTransformer()),
                                                                 ('selectpercentile',
                                                                  SelectPercentile(percentile=95))])),
                                                ('pipeline-2',
                                          

In [175]:
vald_pred = pipeline.predict(df_vald_sp)

In [176]:
accuracy_score(df_train_sp["target"],pipeline.predict(df_train_sp))

0.874085194220304

In [177]:
accuracy_score(df_vald_sp["target"],vald_pred)

0.7964098073555166

In [153]:
subm = pipeline.predict(df_test.text)

KeyError: 'text'

In [ ]:
df_test

In [ ]:
fina_sub= pd.DataFrame(subm, index=df_test.id, columns=["target"])

In [ ]:
final_sub = final_sub.set_index("id")

In [ ]:
fina_sub.to_csv("sub_log.csv")

In [ ]:
kaggle competitions submit -c nlp-getting-started -f sub_log.csv -m "Logistic Regression implemented"